# Week 3 Lab

In this assignment, it is required to explore and cluster the neighborhoods in Toronto.

In [1]:
import numpy as np 
import pandas as pd 

Start by making sure we have a good html-parser.

In [2]:
#!pip install BeautifulSoup4

In [3]:
#!pip install lxml

### Scrape the raw data from Wikipedia.

Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfl = pd.read_html(url, attrs={'class': "wikitable sortable"}, header=0)
dfl[0] # 'cause it's a list of dataframes, even if there's only 1 element in the list.. 

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


### Assigning it to a new DataFrame.

To make it easier to follow.

In [55]:
df = dfl[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


### Only process the cells that have an assigned borough. 

Ignore cells with a borough that is *Not assigned*.

In [6]:
no_bor = df.drop(df[df.Borough == 'Not assigned'].index) 
no_bor.index = range(len(no_bor)) #re-initialize index
no_bor

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


### Combine neighbourhoods with same postcode and borough.


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [63]:
neigh_joined = no_bor.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
neigh_joined

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


### Add borough name if neighbourhood name is not assigned.

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [64]:
#first checking if such even exists
neigh_bor = neigh_joined[neigh_joined.Neighbourhood == 'Not assigned']
neigh_bor

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [65]:
#copying the borough content
neigh_joined.Neighbourhood = np.where(neigh_joined.Neighbourhood == 'Not assigned', neigh_joined.Borough, neigh_joined.Neighbourhood)

In [66]:
#first checking if such even exists
neigh_bor2 = neigh_joined[neigh_joined.Neighbourhood == 'Not assigned']
neigh_bor2

,Postcode,Borough,Neighbourhood


In [67]:
# now verifying even, that the right value got copied and exists:
neigh_bor3 = neigh_joined[neigh_joined.Neighbourhood == 'Queen\'s Park']
neigh_bor3

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park
93,M9A,Downtown Toronto,Queen's Park


Looks like no "not assigned" values left. All cleanups done.

### Print the shape.

In [68]:
neigh_joined.shape

(103, 3)

### Get the latitude and longitude of each postal code.


Trying first tto use the suggested API.

In [69]:
#!pip install geocoder

In [70]:
#import geocoder # import geocoder

In [71]:
### FIRST TEST, FAILS TO RESPOND. REVERTING TO using CSV.
# initialize your variable to None
"""
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M3A'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
"""

"\nlat_lng_coords = None\n\n# loop until you get the coordinates\nwhile(lat_lng_coords is None):\n  g = geocoder.google('{}, Toronto, Ontario'.format('M3A'))\n  lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]\n"

Pulling the CSV instead.

In [72]:
url ='https://cocl.us/Geospatial_data'
geodf = pd.read_csv(url,header=0)
geodf

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [73]:
neigh_postal = neigh_joined.join(geodf.set_index('Postal Code'), on='Postcode')
neigh_postal

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


### Clustering the Toronto neighbourhoods.

Last part of the submission task.

- Explore and cluster the neighborhoods in Toronto. (You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.)

Just make sure:

- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together.

In [74]:
print(neigh_postal.shape)
neigh_postal.head()

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [75]:
toronto = neigh_postal[neigh_postal['Borough'].str.contains('Toronto', regex=False)]
print(toronto.shape)
toronto.head()

(39, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Just checking how many different neighbourhoods we have per borough.

In [85]:
toronto.groupby('Borough').count()

,Postcode,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
West Toronto,6,6,6,6


### Just for fun a Toronto map as well.

In [91]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [93]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [99]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Get the Foursquare API data for venues



In [76]:
# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Define Foursquare Credentials and Version

In [77]:
CLIENT_ID = 'BAL4G1CRPCUBUEOYYEAU5NMI5EVTGVSLJHGBEPW31MCMUMRJ' #  Foursquare ID
CLIENT_SECRET = 'N45SJW35C2ZAMMLFFRAR2JO5FJHB0HEMKGOI0YJ3MGHOIIYZ' #  Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: BAL4G1CRPCUBUEOYYEAU5NMI5EVTGVSLJHGBEPW31MCMUMRJ
CLIENT_SECRET:N45SJW35C2ZAMMLFFRAR2JO5FJHB0HEMKGOI0YJ3MGHOIIYZ


In [78]:
def getTorontoVenues(boroughs, neighbourhoods, latitudes, longitudes, radius=500):
    venues = []
    for borough, neighbourhood, latitude, longitude in zip(boroughs, neighbourhoods, latitudes, longitudes):
        # Defining the URL for FOURSQUARE API 
        print('Getting '+ neighbourhood)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)
           
        # make the request with the URL
        results = requests.get(url).json()
        
        raw_venues = results['response']['groups'][0]['items']

        # tranform venues into a dataframe
        dataframe = json_normalize(raw_venues)
        
        # now selecting only relevant information for each received venue, and keeping the original boroughs data for quicker processing
        for venue in raw_venues:
            venues.append([
            borough, 
            neighbourhood, 
            latitude, 
            longitude, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'],
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']])  
        #print(raw_venues)
    print('All done!')
    return(venues)

### example of json to help figure out the structure
'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '294544',
     'url': 'https://www.seamless.com/menu/harrys-italian-pizza-bar-225-murray-st-new-york/294544?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=294544',
     'provider': {'name': 'seamless',
      'icon': {'prefix': 'https://fastly.4sqi.net/img/general/cap/',
       'sizes': [40, 50],
       'name': '/delivery_provider_seamless_20180129.png'}}},
    'referralId': 'v-1576441767',
    'hasPerk': False},

### Get the Toronto venues information and categories

Running the request to Foursquare API (above) and getting back a list with all the venues.

In [79]:
toronto_venues = getTorontoVenues(boroughs= toronto['Borough'], 
                                  neighbourhoods=toronto['Neighbourhood'],
                                  latitudes=toronto['Latitude'],
                                  longitudes=toronto['Longitude']
                                  )

Getting The Beaches
Getting The Danforth West, Riverdale
Getting The Beaches West, India Bazaar
Getting Studio District
Getting Lawrence Park
Getting Davisville North
Getting North Toronto West
Getting Davisville
Getting Moore Park, Summerhill East
Getting Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Getting Rosedale
Getting Cabbagetown, St. James Town
Getting Church and Wellesley
Getting Harbourfront
Getting Ryerson, Garden District
Getting St. James Town
Getting Berczy Park
Getting Central Bay Street
Getting Adelaide, King, Richmond
Getting Harbourfront East, Toronto Islands, Union Station
Getting Design Exchange, Toronto Dominion Centre
Getting Commerce Court, Victoria Hotel
Getting Roselawn
Getting Forest Hill North, Forest Hill West
Getting The Annex, North Midtown, Yorkville
Getting Harbord, University of Toronto
Getting Chinatown, Grange Park, Kensington Market
Getting CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Land

In [80]:
len(toronto_venues)

835

In [81]:
cols = [ 'Borough',
          'Neighbourhood', 
          'Nghbhd Lat', 
          'Nghbhd Long', 
          'Venue', 
          'Venue Lat', 
          'Venue Long', 
          'Venue Category']

In [82]:
df = pd.DataFrame(toronto_venues, columns = cols) 
df

,Borough,Neighbourhood,Nghbhd Lat,Nghbhd Long,Venue,Venue Lat,Venue Long,Venue Category
0,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,East Toronto,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
...,...,...,...,...,...,...,...,...
830,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
831,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
832,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop
833,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Revolution Recording,43.662561,-79.326940,Recording Studio


In [83]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 4 boroughs and 835 neighborhoods.


In [100]:
print('{} venues were returned by Foursquare.'.format(df.shape[0]))

835 venues were returned by Foursquare.


Checking everything looks as expected with the dataframe. 
Also checking how many venues we got per neighbourhood.

In [103]:
# Now look at how many venues there are per neighbourhood
df.groupby('Neighbourhood')['Venue'].count()

Neighbourhood
Adelaide, King, Richmond                                                                                      30
Berczy Park                                                                                                   30
Brockton, Exhibition Place, Parkdale Village                                                                  22
Business Reply Mail Processing Centre 969 Eastern                                                             16
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara    15
Cabbagetown, St. James Town                                                                                   30
Central Bay Street                                                                                            30
Chinatown, Grange Park, Kensington Market                                                                     30
Christie                                                                          

#### Let's find out how many unique categories can be curated from all the returned venues

In [104]:
print('There are {} uniques categories.'.format(len(df['Venue Category'].unique())))

There are 187 uniques categories.


## Let's analyze each neighbourhood

In [105]:
# one hot encoding
toronto_onehot = pd.get_dummies(df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = df['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
toronto_onehot.shape

(835, 188)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [107]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.033333,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Check again, what's the new size

In [108]:
toronto_grouped.shape

(38, 188)

Let's check how many neighbourhoods are there for each borough.

In [179]:
toronto.groupby('Borough').count()

,Postcode,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
West Toronto,6,6,6,6


How many unique postcodes is that?

In [180]:
print('There are {} uniques postcodes.'.format(len(toronto['Postcode'].unique())))

There are 39 uniques postcodes.


#### Let's print each neighborhood along with the top 5 most common venues

In [109]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0              Café  0.10
1        Steakhouse  0.10
2             Hotel  0.07
3  Asian Restaurant  0.07
4         Speakeasy  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1  Seafood Restaurant  0.07
2            Beer Bar  0.07
3      Farmers Market  0.07
4        Cocktail Bar  0.07


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3   Grocery Store  0.05
4   Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0                   Spa  0.06
1         Auto Workshop  0.06
2            Skate Park  0.06
3         Burrito Place  0.06
4  Fast Food Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0      Airport Lounge  0.13
1     Air

#### Let's put that into a *pandas* dataframe

In [172]:
# first sort the venues descending
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [173]:
# now display top 10 and add to dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Steakhouse,Hotel,Asian Restaurant,Gastropub,Speakeasy,Smoke Shop,Coffee Shop,Seafood Restaurant,Plaza
1,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Farmers Market,Beer Bar,Café,Italian Restaurant,Cheese Shop,Liquor Store,Bistro
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Grocery Store,Restaurant,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection
3,Business Reply Mail Processing Centre 969 Eastern,Pizza Place,Brewery,Burrito Place,Farmers Market,Light Rail Station,Auto Workshop,Restaurant,Spa,Fast Food Restaurant,Recording Studio
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Airport,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden,Boutique


# k-means CLUSTERING!

In [174]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 0], dtype=int32)

In [175]:
# create a new df with clusters and top venues
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Nghbhd Lat,Nghbhd Long,Venue,Venue Lat,Venue Long,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail,0,Other Great Outdoors,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Coworking Space,Dog Run,Discount Store,Diner
1,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store,0,Other Great Outdoors,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Coworking Space,Dog Run,Discount Store,Diner
2,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub,0,Other Great Outdoors,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Coworking Space,Dog Run,Discount Store,Diner
3,East Toronto,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors,0,Other Great Outdoors,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Coworking Space,Dog Run,Discount Store,Diner
4,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood,0,Other Great Outdoors,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Coworking Space,Dog Run,Discount Store,Diner


## Time to visualize the clusters

In [176]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Venue Lat'], toronto_merged['Venue Long'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Taking a closer look on the clusters

## Cluster 1

In [177]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Venue Lat,Venue Long,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,43.676821,-79.293942,Trail,0,Other Great Outdoors,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Coworking Space,Dog Run,Discount Store,Diner
1,The Beaches,43.678879,-79.297734,Health Food Store,0,Other Great Outdoors,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Coworking Space,Dog Run,Discount Store,Diner
2,The Beaches,43.679181,-79.297215,Pub,0,Other Great Outdoors,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Coworking Space,Dog Run,Discount Store,Diner
3,The Beaches,43.676300,-79.294784,Other Great Outdoors,0,Other Great Outdoors,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Coworking Space,Dog Run,Discount Store,Diner
4,The Beaches,43.680563,-79.292869,Neighborhood,0,Other Great Outdoors,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Coworking Space,Dog Run,Discount Store,Diner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,Business Reply Mail Processing Centre 969 Eastern,43.664908,-79.322560,Light Rail Station,0,Pizza Place,Brewery,Burrito Place,Farmers Market,Light Rail Station,Auto Workshop,Restaurant,Spa,Fast Food Restaurant,Recording Studio
831,Business Reply Mail Processing Centre 969 Eastern,43.664815,-79.324213,Spa,0,Pizza Place,Brewery,Burrito Place,Farmers Market,Light Rail Station,Auto Workshop,Restaurant,Spa,Fast Food Restaurant,Recording Studio
832,Business Reply Mail Processing Centre 969 Eastern,43.664538,-79.325379,Smoke Shop,0,Pizza Place,Brewery,Burrito Place,Farmers Market,Light Rail Station,Auto Workshop,Restaurant,Spa,Fast Food Restaurant,Recording Studio
833,Business Reply Mail Processing Centre 969 Eastern,43.662561,-79.326940,Recording Studio,0,Pizza Place,Brewery,Burrito Place,Farmers Market,Light Rail Station,Auto Workshop,Restaurant,Spa,Fast Food Restaurant,Recording Studio


## Cluster 2

In [178]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Venue Lat,Venue Long,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,Studio District,43.660656,-79.342019,Ice Cream Shop,1,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Coworking Space,Middle Eastern Restaurant,Ice Cream Shop,Comfort Food Restaurant,Neighborhood
57,Studio District,43.660892,-79.340626,Sandwich Place,1,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Coworking Space,Middle Eastern Restaurant,Ice Cream Shop,Comfort Food Restaurant,Neighborhood
58,Studio District,43.660651,-79.342267,Bookstore,1,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Coworking Space,Middle Eastern Restaurant,Ice Cream Shop,Comfort Food Restaurant,Neighborhood
59,Studio District,43.661373,-79.338577,Coffee Shop,1,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Coworking Space,Middle Eastern Restaurant,Ice Cream Shop,Comfort Food Restaurant,Neighborhood
60,Studio District,43.660407,-79.343257,Fish Market,1,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Coworking Space,Middle Eastern Restaurant,Ice Cream Shop,Comfort Food Restaurant,Neighborhood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,"Runnymede, Swansea",43.649024,-79.484928,Gastropub,1,Café,Coffee Shop,Italian Restaurant,Sushi Restaurant,Falafel Restaurant,Food,Dessert Shop,Bookstore,Sandwich Place,Burrito Place
815,"Runnymede, Swansea",43.649937,-79.482366,Food & Drink Shop,1,Café,Coffee Shop,Italian Restaurant,Sushi Restaurant,Falafel Restaurant,Food,Dessert Shop,Bookstore,Sandwich Place,Burrito Place
816,"Runnymede, Swansea",43.649395,-79.483560,Pizza Place,1,Café,Coffee Shop,Italian Restaurant,Sushi Restaurant,Falafel Restaurant,Food,Dessert Shop,Bookstore,Sandwich Place,Burrito Place
817,"Runnymede, Swansea",43.650173,-79.481635,Cosmetics Shop,1,Café,Coffee Shop,Italian Restaurant,Sushi Restaurant,Falafel Restaurant,Food,Dessert Shop,Bookstore,Sandwich Place,Burrito Place


## Cluster 3

In [179]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Venue Lat,Venue Long,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
503,Roselawn,43.710634,-79.415810,Health & Beauty Service,2,Health & Beauty Service,Garden,Yoga Studio,Creperie,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
504,Roselawn,43.712189,-79.411978,Garden,2,Health & Beauty Service,Garden,Yoga Studio,Creperie,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


## Cluster 4

In [180]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Venue Lat,Venue Long,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
151,"Moore Park, Summerhill East",43.690350,-79.383313,Intersection,3,Restaurant,Playground,Intersection,Creperie,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
152,"Moore Park, Summerhill East",43.692816,-79.384504,Restaurant,3,Restaurant,Playground,Intersection,Creperie,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
153,"Moore Park, Summerhill East",43.693610,-79.383465,Playground,3,Restaurant,Playground,Intersection,Creperie,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega


## Cluster 5

In [181]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Venue Lat,Venue Long,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
169,Rosedale,43.682328,-79.378934,Playground,4,Park,Playground,Trail,Coworking Space,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
170,Rosedale,43.682036,-79.373788,Park,4,Park,Playground,Trail,Coworking Space,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
171,Rosedale,43.678300,-79.382773,Park,4,Park,Playground,Trail,Coworking Space,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
172,Rosedale,43.676352,-79.373842,Trail,4,Park,Playground,Trail,Coworking Space,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
505,"Forest Hill North, Forest Hill West",43.700726,-79.410101,Trail,4,Park,Jewelry Store,Trail,Sushi Restaurant,Creperie,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
506,"Forest Hill North, Forest Hill West",43.697945,-79.406605,Park,4,Park,Jewelry Store,Trail,Sushi Restaurant,Creperie,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
507,"Forest Hill North, Forest Hill West",43.700443,-79.407957,Sushi Restaurant,4,Park,Jewelry Store,Trail,Sushi Restaurant,Creperie,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
508,"Forest Hill North, Forest Hill West",43.700374,-79.407644,Jewelry Store,4,Park,Jewelry Store,Trail,Sushi Restaurant,Creperie,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


_Author: Ele-Kaja Gildemann 2019_